In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from flatsurf import *
import numpy as np
from matplotlib import pyplot as plt
import os
import pwlf
import os
from surface_dynamics.all import *
from Library import *
from Library import Section
import math
from time import time
import copy
from scipy import integrate
import sympy as sym
from sympy import Symbol
from sympy import solve, lambdify
import traceback
import dill
import sys

In [2]:
# list of permutations
def perms_list(n, **kwargs):
    obstructed = []
    p = D.query(nb_squares=n, **kwargs)
    for item in p:
        if not is_unobstructed(item):
            obstructed.append(item)
            for perm in item.teichmueller_curve():
                obstructed.append(perm)
    return obstructed

In [3]:
# take files from saddle.py and load them into notebook
def load_arrays_from_file(file_path):
    # Load arrays from the NumPy file
    arrays_list = np.load(file_path, allow_pickle=True)
    
    # Ensure each element in the list is a NumPy array
    arrays_list = [np.array(array) for array in arrays_list]
    
    return arrays_list

In [4]:
# 7-0, 7-3, 7-7, 7-19, 7-29, 7-41
n_squares = 7
index = 0
j = 0
dx = 0.0005

In [5]:
permutations = perms_list(n_squares)
perm = permutations[index]

vec_file = "vecs" + str(n_squares) + "-" + str(index) + ".npy"
vecs0 = load_arrays_from_file(os.path.join("vecs", vec_file))

with open(os.path.join("results", f"{n_squares} - {index}", "setup.dill"), 'rb') as f:
    loaded_data = dill.load(f)
a,c,e,g = loaded_data

In [6]:
vecs0 = vecs0[:1000000]
len(vecs0)

1000000

In [9]:
#go thorugh all cusps
dfs = []
dfs1 = []
for j in range(len(a[0])):

    #change back to False in future
    improved = False
    if j == 0:
        improved = True
        
    for i in range(len(a)):

        #get dimensions of section 
        vecs, x_vals, m0, m1, x0, y0, dx_y, z = setup(a[i][j], c[i][j], e[i][j], vecs0, dx, improved)

        if float(z) <= float(1/50000):
            print("too small")
            continue

        print("new")
        t0 = time()
        dfs1.append(winners1(vecs, x_vals, m0, m1, y0, dx, dx_y))
        t1 = time()
        print(t1-t0)
        print()
        print("old")
        t2 = time()
        dfs.append(winners(vecs, x_vals, m0, m1, y0, dx, dx_y))
        t3 = time()
        print(t3-t2)
        print()
        break

new
top done: 1.1243491172790527
side done: 1.1910438537597656
70.76596522331238

old
top done: 334.55380940437317
side done: 326.82806396484375
727.8051013946533

new
top done: 1.1188580989837646
side done: 0.8689651489257812
37.42774510383606

old
top done: 334.6411409378052
side done: 318.82910108566284
688.0716397762299

new
top done: 1.2444727420806885
side done: 0.8673231601715088
37.890198707580566

old
top done: 338.2408893108368
side done: 319.87397718429565
693.0178265571594

new
top done: 1.1096012592315674
side done: 0.8865048885345459
37.7015745639801

old
top done: 333.27572107315063
side done: 317.02412152290344
685.3480386734009

new
top done: 1.1045961380004883
side done: 0.8971529006958008
37.68028497695923

old
top done: 333.2941975593567
side done: 322.39687991142273
690.96497631073

new
top done: 1.0953645706176758
side done: 0.8758354187011719
37.47906684875488

old
top done: 333.4045968055725
side done: 318.80995512008667
687.1966960430145

new
top done: 1.095360

In [59]:
df = dfs1[0]
df1 = dfs[0]
df.iloc[[int(499673)]], df1.iloc[[int(499673)]]

(            x      y    lab   vec  time
 499673  0.501 -0.001  50000  None  1000,
             x      y    lab   vec  time
 499673  0.501 -0.001  50000  None  1000)

In [ ]:
for j in range(len(dfs)):
    df1 = dfs[j]
    df2 = dfs1[j]
    
    for i in range(len(df1)):
        for col in df1.columns:
            if col == "vec":
                if np.array_equal(df1.loc[i, col], df2.loc[i, col]):
                    continue
                else:
                    print(i)
                    break
            else: 
                if df1.loc[i, col] == df2.loc[i, col]:
                    continue
                else: 
                    print(i)
                    break
    print(str(j) + " done")

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done


In [7]:
def winners1(vecs0, x_vals, m0, m1, y0, dx, dx_y):
    # dictionary for plotting
    saddle_dict = {}
    saddle_dict["x"] = []
    saddle_dict["y"] = []
    saddle_dict["lab"] = []
    saddle_dict["vec"] = []
    saddle_dict["time"] = []
    possible_vecs = []
    winners = []
    dz = 1 / 100
    vecs = np.hstack(vecs0)  # Each vec is a column in this 2D array
    
    # top edge
    # Stack list of numpy arrays into a single 2D array
    t0 = time()
    for a in np.arange(dz, 1, dz):
        Mab = np.array([[a, 1 - dx / y0], [0, a]])  # Matrix stays outside the inner loop
    
        # Apply the transformation to all vectors at once
        new_vecs = Mab @ vecs
    
        # Extract x and y components
        x_comps = new_vecs[0, :]
        y_comps = new_vecs[1, :]
    
        # Filter based on conditions (x > 0, y/x > 0, and x <= 1)
        valid_mask = (x_comps > 0) & (x_comps <= 1) & (y_comps / x_comps > 0)
    
        valid_x = x_comps[valid_mask]
        valid_y = y_comps[valid_mask]
        valid_vecs = vecs[:, valid_mask]  # Apply the mask to filter valid vectors
    
        if valid_x.size == 0:
            winners.append(None)
            continue
    
        # Calculate slopes
        slopes = valid_y / valid_x
    
        # Find the minimum slope and handle continuity cases
        min_slope_idx = np.argmin(slopes)
        winner_slope = slopes[min_slope_idx]
        winner = valid_vecs[:, min_slope_idx]
    
        # Handle continuity by finding the smallest vector if slopes are close
        for i, slope in enumerate(slopes):
            if np.abs(slope - winner_slope) <= dx / 1000:
                if valid_vecs[0, i] < winner[0] or valid_vecs[1, i] < winner[1]:
                    winner = valid_vecs[:, i]
    
        winners.append(winner.reshape(2,1))
    t1 = time()
    print("top done: " + str(t1 - t0))
    
    # diagonal
    #for a in np.arange(0 + dz, 1, dz):
        #Mab = np.array([[a, m1*a + 1-dx/y0 + dx_y], [0, a]])
        # Apply the transformation to all vectors at once
        #new_vecs = Mab @ vecs
    
        # Extract x and y components
        #x_comps = new_vecs[0, :]
        #y_comps = new_vecs[1, :]
    
        # Filter based on conditions (x > 0, y/x > 0, and x <= 1)
        #valid_mask = (x_comps > 0) & (x_comps <= 1) & (y_comps / x_comps > 0)
    
        #valid_x = x_comps[valid_mask]
        #valid_y = y_comps[valid_mask]
        #valid_vecs = vecs[:, valid_mask]  # Apply the mask to filter valid vectors
    
        #if valid_x.size == 0:
            #winners.append(None)
            #continue
    
        # Calculate slopes
        #slopes = valid_y / valid_x
    
        # Find the minimum slope and handle continuity cases
        #min_slope_idx = np.argmin(slopes)
        #winner_slope = slopes[min_slope_idx]
        #winner = valid_vecs[:, min_slope_idx]
    
        # Handle continuity by finding the smallest vector if slopes are close
        #for i, slope in enumerate(slopes):
            #if np.abs(slope - winner_slope) <= dx / 1000:
                #if valid_vecs[0, i] < winner[0] or valid_vecs[1, i] < winner[1]:
                    #winner = valid_vecs[:, i]
    
        #winners.append(winner.reshape(2,1))

    # side edge
    t0 = time()
    y_vals = np.arange(m1 + (1-dx)/y0 + dx_y, m0 +
                       (1-dx)/y0 - dx_y, dz*(m0-m1))
    for b in y_vals:
        Mab = np.array([[1 - dx, b], [0, 1-dx]])
        # Apply the transformation to all vectors at once
        new_vecs = Mab @ vecs
    
        # Extract x and y components
        x_comps = new_vecs[0, :]
        y_comps = new_vecs[1, :]
    
        # Filter based on conditions (x > 0, y/x > 0, and x <= 1)
        valid_mask = (x_comps > 0) & (x_comps <= 1) & (y_comps / x_comps > 0)
    
        valid_x = x_comps[valid_mask]
        valid_y = y_comps[valid_mask]
        valid_vecs = vecs[:, valid_mask]  # Apply the mask to filter valid vectors
    
        if valid_x.size == 0:
            winners.append(None)
            continue
    
        # Calculate slopes
        slopes = valid_y / valid_x
    
        # Find the minimum slope and handle continuity cases
        min_slope_idx = np.argmin(slopes)
        winner_slope = slopes[min_slope_idx]
        winner = valid_vecs[:, min_slope_idx]
    
        # Handle continuity by finding the smallest vector if slopes are close
        for i, slope in enumerate(slopes):
            if np.abs(slope - winner_slope) <= dx / 1000:
                if valid_vecs[0, i] < winner[0] or valid_vecs[1, i] < winner[1]:
                    winner = valid_vecs[:, i]
        winners.append(winner.reshape(2,1))
    t1 = time()
    print("side done: " + str(t1 - t0))
    
    winners2 = []
    for winner in winners:
        try:
            if winner == None:
                continue
        except:
            winners2.append(winner)
    possible_vecs1 = np.unique(winners2, axis=0)
    for item in possible_vecs1:
        possible_vecs.append(item)

    global label_dict
    label_dict = {}

    # dictionary for vector labels
    for i in range(len(possible_vecs)):
        label_dict[i] = possible_vecs[i]

    # for each vector, there is a time function defined as f(a,b) where a,b are points in the poincare section
    global t_dict
    t_dict = {}

    x, y, t = sym.symbols('x y t')
    Mab = np.array([[x, y], [0, 1/x]])
    horo = np.array([[1, 0], [-t, 1]])

    for i in range(len(possible_vecs)):
        # apply Mab matrix, perform horocycle flow and find time t to horizontal
        a = horo@(Mab@possible_vecs[i])
        t_dict[i] = lambdify([x, y], solve(a[1][0], t)[0])

    # for each point (a,b) in the poincare section, apply the Mab matrix to each vector and look for "winners". Winners have smallest possible slope that is greater than zero and 0 < x-component <= 1
    for a in x_vals:
        y_vals = np.arange(m1*a + 1/y0 + dx_y, m0*a + 1/y0 - dx_y, dx_y)
        for b in y_vals:
            check = 0
            winner_slope = None
            winner = None
            Mab = np.array([[a, b], [0, 1/a]])
            for vec in possible_vecs:
                new = Mab@vec
                if float(new[0][0]) == 0:
                    continue
                x = float(new[0][0])
                y = float(new[1][0])
                if y/x <= 0:
                    continue
                if x <= 1 and x > 0:
                    if winner_slope == None:
                        winner_slope = y/x
                        winner = vec
                        continue
            # if you have two potential winners like (m,n) and 2*(m,n), make (m,n) winner for continuity and plotting purposes
                    elif abs(y/x - winner_slope) <= dx/1000:
                        if vec[0][0] < winner[0][0] or vec[1][0] < winner[1][0]:
                            winner = vec
                            continue
                    elif y/x < winner_slope:
                        winner_slope = y/x
                        winner = vec
                        continue

            saddle_dict["x"].append(a)
            saddle_dict["y"].append(b)
            saddle_dict["vec"].append(winner)
            for i in range(len(possible_vecs)):
                if np.array_equal(winner, possible_vecs[i]):
                    check += 1
                    saddle_dict["lab"].append(i)
                    saddle_dict["time"].append(t_dict[i](a, b))
                    if saddle_dict["time"][-1] < 0:
                        saddle_dict["time"][-1] = 1000
            # if there is no winner, at (a,b), add a label so the df and plot can still be made. These section will later be made blank for trouble-shoooting
            if check == 0:
                saddle_dict["lab"].append(len(vecs0))
                saddle_dict["time"].append(1000)

    df = pd.DataFrame.from_dict(saddle_dict)
    return df



In [8]:
def winners(vecs, x_vals, m0, m1, y0, dx, dx_y):
    # dictionary for plotting
    saddle_dict = {}
    saddle_dict["x"] = []
    saddle_dict["y"] = []
    saddle_dict["lab"] = []
    saddle_dict["vec"] = []
    saddle_dict["time"] = []
    possible_vecs = []
    winners = []

    # top edge
    t0 = time()
    dz = 1/100
    for a in np.arange(dz, 1, dz):
        check = 0
        winner_slope = None
        winner = None
        Mab = np.array([[a, 1-dx/y0], [0, a]])
        for vec in vecs:
            new = Mab@vec
            if float(new[0][0]) == 0:
                continue
            x = float(new[0][0])
            y = float(new[1][0])
            if y/x <= 0:
                continue
            if x <= 1 and x > 0:
                if winner_slope == None:
                    winner_slope = y/x
                    winner = vec
                    continue
        # if you have two potential winners like (m,n) and 2*(m,n), make (m,n) winner for continuity and plotting purposes
                elif abs(y/x - winner_slope) <= dx/1000:
                    if vec[0][0] < winner[0][0] or vec[1][0] < winner[1][0]:
                        winner = vec
                        continue
                elif y/x < winner_slope:
                    winner_slope = y/x
                    winner = vec
                    continue
        winners.append(winner)
    t1 = time()
    print("top done: " + str(t1 - t0))

    # diagonal
    #for a in np.arange(0 + dz, 1, dz):
        #check = 0
        #winner_slope = None
        #winner = None
        #Mab = np.array([[a, m1*a + 1-dx/y0 + dx_y], [0, a]])
        #for vec in vecs:
            #new = Mab@vec
            #if float(new[0][0]) == 0:
                #continue
            #x = float(new[0][0])
            #y = float(new[1][0])
            #if y/x <= 0:
                #continue
            #if x <= 1 and x > 0:
                #if winner_slope == None:
                    #winner_slope = y/x
                    #winner = vec
                    #continue
       # if you have two potential winners like (m,n) and 2*(m,n), make (m,n) winner for continuity and plotting purposes
                #elif abs(y/x - winner_slope) <= dx/1000:
                    #if vec[0][0] < winner[0][0] or vec[1][0] < winner[1][0]:
                        #winner = vec
                        #continue
                #elif y/x < winner_slope:
                    #winner_slope = y/x
                    #winner = vec
                    #continue
        #winners.append(winner)

    # side edge
    t0 = time()
    y_vals = np.arange(m1 + (1-dx)/y0 + dx_y, m0 +
                       (1-dx)/y0 - dx_y, dz*(m0-m1))
    for b in y_vals:
        check = 0
        winner_slope = None
        winner = None
        Mab = np.array([[1 - dx, b], [0, 1-dx]])
        for vec in vecs:
            new = Mab@vec
            if float(new[0][0]) == 0:
                continue
            x = float(new[0][0])
            y = float(new[1][0])
            if y/x <= 0:
                continue
            if x <= 1 and x > 0:
                if winner_slope == None:
                    winner_slope = y/x
                    winner = vec
                    continue
        # if you have two potential winners like (m,n) and 2*(m,n), make (m,n) winner for continuity and plotting purposes
                elif abs(y/x - winner_slope) <= dx/1000:
                    if vec[0][0] < winner[0][0] or vec[1][0] < winner[1][0]:
                        winner = vec
                        continue
                elif y/x < winner_slope:
                    winner_slope = y/x
                    winner = vec
                    continue
        winners.append(winner)
    t1 = time()
    print("side done: " + str(t1 - t0))
    
    winners2 = []
    for winner in winners:
        try:
            if winner == None:
                continue
        except:
            winners2.append(winner)
    possible_vecs1 = np.unique(winners2, axis=0)
    for item in possible_vecs1:
        possible_vecs.append(item)

    global label_dict
    label_dict = {}

    # dictionary for vector labels
    for i in range(len(possible_vecs)):
        label_dict[i] = possible_vecs[i]

    # for each vector, there is a time function defined as f(a,b) where a,b are points in the poincare section
    global t_dict
    t_dict = {}

    x, y, t = sym.symbols('x y t')
    Mab = np.array([[x, y], [0, 1/x]])
    horo = np.array([[1, 0], [-t, 1]])

    for i in range(len(possible_vecs)):
        # apply Mab matrix, perform horocycle flow and find time t to horizontal
        a = horo@(Mab@possible_vecs[i])
        t_dict[i] = lambdify([x, y], solve(a[1][0], t)[0])

    # for each point (a,b) in the poincare section, apply the Mab matrix to each vector and look for "winners". Winners have smallest possible slope that is greater than zero and 0 < x-component <= 1
    for a in x_vals:
        y_vals = np.arange(m1*a + 1/y0 + dx_y, m0*a + 1/y0 - dx_y, dx_y)
        for b in y_vals:
            check = 0
            winner_slope = None
            winner = None
            Mab = np.array([[a, b], [0, 1/a]])
            for vec in possible_vecs:
                new = Mab@vec
                if float(new[0][0]) == 0:
                    continue
                x = float(new[0][0])
                y = float(new[1][0])
                if y/x <= 0:
                    continue
                if x <= 1 and x > 0:
                    if winner_slope == None:
                        winner_slope = y/x
                        winner = vec
                        continue
            # if you have two potential winners like (m,n) and 2*(m,n), make (m,n) winner for continuity and plotting purposes
                    elif abs(y/x - winner_slope) <= dx/1000:
                        if vec[0][0] < winner[0][0] or vec[1][0] < winner[1][0]:
                            winner = vec
                            continue
                    elif y/x < winner_slope:
                        winner_slope = y/x
                        winner = vec
                        continue

            saddle_dict["x"].append(a)
            saddle_dict["y"].append(b)
            saddle_dict["vec"].append(winner)
            for i in range(len(possible_vecs)):
                if np.array_equal(winner, possible_vecs[i]):
                    check += 1
                    saddle_dict["lab"].append(i)
                    saddle_dict["time"].append(t_dict[i](a, b))
                    if saddle_dict["time"][-1] < 0:
                        saddle_dict["time"][-1] = 1000
            # if there is no winner, at (a,b), add a label so the df and plot can still be made. These section will later be made blank for trouble-shoooting
            if check == 0:
                saddle_dict["lab"].append(len(vecs))
                saddle_dict["time"].append(1000)

    df = pd.DataFrame.from_dict(saddle_dict)
    return df
